## Assignment 2

*100 points (8% of course grade)*</br>
*Assigned: Wed, Jun 4th*</br>
**Due: Mon, Jun 16th, 23:59**

This homework should be done in parts as soon as (< 1 week) relevant topics are covered in lectures. If you wait until the last minute, you might be overwhelmed.

You must turn in the required files electronically, including this Notebook (A2.ipynb). Please follow the submission instructions for each problem carefully.

In this assignment, you need to solve two problems. In Problem 1, you will write a few SQL queries to query the beers database (the same as in A1). In Problem 2, you will answer two questions in database design theory.

## Setup your PostgreSQL

You will need this setup to create a database on your machine and to test your queries. Please follow our [setup instructions](https://canvas.sfu.ca/courses/84701/pages/postgresql-setup-instruction-for-assignment-2) on Canvas.

### Problem 1: Query with SQL (64%)

Consider again the beer drinker's database from Assignment 1 with following schema (key columns underlined).

- drinker (<u>name</u>, address)
- bar (<u>name</u>, address)
- beer (<u>name</u>, brewer)
- frequents (<u>drinker</u>, <u>bar</u>, times_a_week)
- likes (<u>drinker</u>, <u>beer</u>)
- serves (<u>bar</u>, <u>beer</u>, price)


#### **Preliminary**

After you finish the PostgreSQL setup, you will be able to run the PostgreSQL's interpreter in your command line tools by running `psql beers` in your terminal or command line. You may also use pgAdmin instead of the command line tool.

Same as assignment 1, you will need to type your SQL queries in the cells below.


Now your homework question is to write Relational Algebra queries to answer following questions. 

Please fill your answer in each cell (and **ONLY the query**) and **DO NOT add or remove** any cells to make the TAs' life easier in evaluating your queries. Questions (1)-(4) are worth 6 points each; (5)-(9) are worth 8 points each.


#### 0. (example) Find names of all bars that Eve frequents.

/* input your answer in this cell: */

SELECT bar FROM frequents WHERE drinker = 'Eve';

#### 1. Find names of bars that serve either Amstel or Corona at price higher than \\$2.

In [ ]:
/* input your answer in this cell: */

SELECT DISTINCT bar FROM Serves WHERE (beer = 'Amstel' OR beer = 'Corona') AND price > 2; 

#### 2. Find the names of all drinkers that like Corona but frequent no bars that serve Corona.

In [ ]:
/* input your answer in this cell: */

SELECT DISTINCT drinker FROM likes L
WHERE L.beer = 'Corona' 
AND NOT EXISTS(
    SELECT * FROM Serves S NATURAL JOIN Frequents F 
    WHERE S.beer = 'Corona' AND F.drinker = L.drinker
);

#### 3. Find the names of all bars that serve at least 5 beers

In [ ]:

SELECT bar FROM Serves
GROUP BY bar
HAVING COUNT(beer) >= 5;

/* input your answer in this cell: */


#### 4. Find the pair of drinkers who frequent bars the same total number of times per week. Don't list (drinkerA, drinkerA). Only list pairs (drinkerA, drinkerB) where drinkerA < drinkerB  in the answer; don't list (drinkerB, drinkerA) again.


In [ ]:
/* input your answer in this cell: */
SELECT f1.drinker, f2.drinker
FROM (SELECT drinker, sum(times_a_week) as st FROM frequents GROUP BY drinker) f1,
 (SELECT drinker, sum(times_a_week) as st FROM frequents GROUP BY drinker) f2
WHERE f1.drinker < f2.drinker AND f1.st = f2.st

#### 5. Find names of all drinkers who frequent *only* those bars that serve *some* beers they like.

In [ ]:
/* input your answer in this cell: */

SELECT name 
FROM drinker 
WHERE NOT EXISTS (
    SELECT bar 
    FROM frequents 
    WHERE drinker = name 
    AND bar NOT IN (
        SELECT bar 
        FROM serves, likes 
        WHERE drinker = name 
        AND serves.beer = likes.beer)
);

#### 6. Find the name of each beer such that in every bar where it is served, it is liked by some drinker who frequents this bar. (If a beer is served nowhere, it should be returned as well.)

In [ ]:
/* input your answer in this cell: */
SELECT name
FROM Beer
WHERE NOT EXISTS (
    SELECT * FROM Serves s
    WHERE s.beer = Beer.name AND NOT EXISTS (
        SELECT * FROM  Frequents f, Likes l
        WHERE s.bar = f.bar AND f.drinker = l.drinker AND l.beer = Beer.name
    )
);

#### 7. Calculate some statistics for each bar: (1) total number of drinkers who frequent it, (2) average price of beers it serves, and (3) name of the drinker who frequents it the maximum number of times a week (the most regular customer!). Your query should output bar, avg_price,
In case of ties in (3), rank the drinkers by the alphabetical order or their names. Sort the output by the number of drinkers (in descending order), in case of ties,
- sort by bar in alphabetical order. You need to list every bar, even if it
is not frequented by anyone
- (show 0 as the total number of drinkers in this case and NULL as the most
regular customer) or
- serves no beers (show NULL as average price in this case).

In [ ]:
/* input your answer in this cell: */
SELECT b.name,
(SELECT GREATEST(COUNT(DISTINCT drinker), 0)
FROM frequents
WHERE bar=b.name) AS num_drinker,
(SELECT AVG(price)
FROM serves s
WHERE bar = b.name) AS avg_price,
(SELECT drinker
FROM frequents
WHERE bar=b.name
ORDER BY times_a_week DESC, drinker ASC
LIMIT 1) AS most_reg_drinker
FROM bar b
ORDER BY num_drinker DESC, b.name ASC;

#### 8. Find all (bar1, bar2) pairs where: (1) at least one same beer is served, and (2) the average number of visits per drinker (exclude those who do not frequent) per week of bar1 is more than 50% higher than the average number of visits per drinker of bar2.


In [ ]:
/* input your answer in this cell: */

SELECT DISTINCT s1.bar, s2.bar
FROM serves s1, serves s2
WHERE s1.bar <> s2.bar
AND s1.beer = s2.beer
AND ((SELECT avg(times_a_week) FROM frequents f1 WHERE s1.bar = f1.bar) > 1.5*(SELECT avg(times_a_week) FROM frequents f2 WHERE s2.bar = f2.bar));


#### 9. Find, for each beer, its lowest serving price and the bar(s) serving it at this price. The output should be a list of (beer, price, bar) triples. If some beer B is not served anywhere, you should still output (B, NULL, NULL).


In [ ]:
/* input your answer in this cell: */

SELECT beer.name, min_price, bar 
FROM beer LEFT OUTER JOIN 
    (SELECT beer, MIN(price) AS min_price FROM serves GROUP BY beer) t 
ON beer.name = t.beer 
LEFT OUTER JOIN serves 
ON beer.name = serves.beer AND min_price = serves.price

## Problem 2: Database design (36% = 12% + 24%)

#### 1. Consider a relation R with five attributes ABCDE. You are given the following dependencies: A -> D; DE -> C; CB -> A. 
1. For each FD $X \rightarrow Y$, compute the closure $X^+$
2. List all keys in R.
3. Is R in 3NF? Explain.
4. Is R in BCNF? Explain.

`Write your answers for 1,2,3,4 in this cell`

1. $A^+ = \{AD\}$, ${DE}^+ = \{CDE\}$, $\{CB\}^+ = \{ABCD\}$

2. {CBE} {ABE} {DEB}

3. Yes. All attributes not appearing on the RHS of any FD (B and E) are included in all keys.

4. Consider A -> D. Since A+ = AD, A is not a superkey and thus A -> D is a BCNF violation.

#### 2. Consider the following table storing information about Pals, jobs, and production structures in a simplified version of Palworld: R(pid, sid, species, structure_name, work_speed, job_type, food_consumption) and a set of functional dependencies:
* FD1: species, structure_name -> work_speed
* FD2: sid -> structure_name, job_type
* FD3: pid -> species, food_consumption
* FD4: structure_name -> job_type

Decompose the schema into BCNF by (1) filling out the steps below, and (2) writing the names of the
relations in the final solution. Note: You may not need all four steps in your decompositions.

#### *Note that this is not the only solution, you may apply FDs in different orders.*

**Step 1**
- Initial table: `R(pid, sid, species, structure_name, work_speed, job_type, food_consumption)`
- Violating FD $X \rightarrow Y$: FD2
- Closure $X^+$ for violating FD $X \rightarrow Y$: {sid, structure_name, job_type}
- Schema of new table-1: `R1(sid, structure_name, job_type)`
- Schema of new table-2: `R2(pid, sid, species, work_speed, food_consumption)`

**Step 2**
- Initial table: R1
- Violating FD $X \rightarrow Y$: FD4
- Closure $X^+$ for violating FD $X \rightarrow Y$: {structure_name, job_type}
- Schema of new table-1: `R3(sid, structure_name)`
- Schema of new table-2: `R4(structure_name, job_type)`

**Step 3**
- Initial table: R2
- Violating FD $X \rightarrow Y$: FD3
- Closure $X^+$ for violating FD $X \rightarrow Y$: {pid, species, food_consumption}
- Schema of new table-1: `R5(pid, species, food_consumption)`
- Schema of new table-2: `R6(pid, sid, work_speed)`

**Step 4**
- Initial table: 
- Violating FD $X \rightarrow Y$:
- Closure $X^+$ for violating FD $X \rightarrow Y$:
- Schema of new table-1:
- Schema of new table-2:

### Submission instruction

1. For problem 1, answer the questions (1)-(9) in the Markdown cells.

2. For problem 2, fill the steps you need after the `:`.

3. Compress your A2.ipynb (this file) into A2.zip and submit on Canvas.